In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pathfinder-hard-integer/merged_data.h5
/kaggle/input/pathfinder-easy-integer/merged_data.h5
/kaggle/input/pathfinder-medium-integer/merged_data.h5


In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.functional import accuracy
import h5py

torch.manual_seed(19)

In [3]:
# Dataset
class SequenceDataset(Dataset):
    def __init__(self, h5_file):
        self.h5_file = h5py.File(h5_file, "r")
        self.images = self.h5_file["images"]  # Shape: (N, 32, 32)
        self.labels = self.h5_file["labels"]  # Shape: (N,)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Get image and reshape to a sequence of 1024 pixels
        image = self.images[idx]  # Shape: (32, 32)
        image = image.flatten()  # Shape: (1024,)
        image = torch.tensor(image, dtype=torch.int).unsqueeze(-1)  # Shape: (1024, 1)
        
        # Get label
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        return image, label

In [4]:
class RNNNormalizationModel(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=128, num_layers=1, output_dim=1):
        super().__init__()
        # RNN Layer
        self.rnn = nn.RNN(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.float() / 255.0  # Normalize inputs
        _, hidden_state = self.rnn(x)  # RNN layer
        output = self.fc(hidden_state.squeeze())  # Fully connected layer
        return output


# LSTM Model
class LSTMNormalizationModel(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=128, num_layers=1, output_dim=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Normalize input
        x = x.float() / 255.0

        # LSTM layer
        hidden_state = self.lstm(x)[1][0].squeeze()

        # Fully connected layer (use last hidden state)
        output = self.fc(hidden_state)  # Shape: [batch_size, output_dim]
        return output


# GRU Model
class GRUNormalizationModel(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=128, num_layers=1, output_dim=1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Normalize input
        x = x.float() / 255.0

        # GRU layer
        _, hidden_state = self.gru(x)

        # Fully connected layer (use last hidden state)
        output = self.fc(hidden_state.squeeze())  # Shape: [batch_size, output_dim]
        return output

In [5]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size=256, input_dim=64, hidden_dim=128, num_layers=1, output_dim=1):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_dim)
        
        # RNN Layer
        self.rnn = nn.RNN(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x).squeeze(2)  # Convert indices to embeddings
        _, hidden_state = self.rnn(x)  # RNN layer
        output = self.fc(hidden_state.squeeze())  # Fully connected layer
        return output


# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size=256, input_dim=64, hidden_dim=128, num_layers=1, output_dim=1):
        super().__init__()

        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        e = self.embedding(x).squeeze(2)
        lstm_out = self.lstm(e)[1][0].squeeze()
        output = self.fc(lstm_out)  # Fully connected layer
        return output


class GRUModel(nn.Module):
    def __init__(self, vocab_size=256, input_dim=64, hidden_dim=128, num_layers=1, output_dim=1):
        super(GRUModel, self).__init__()

        # Embedding
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_dim)
        
        # GRU Layer
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        e = self.embedding(x).squeeze(2)

        # Pass through GRU
        _, out = self.gru(e)
        
        # Fully connected layer
        out = self.fc(out.squeeze())
        return out


In [6]:
class SequenceClassifier(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.model = model
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.learning_rate = learning_rate
        self.train_losses = []
        self.val_losses = []
        self.val_outputs = []  # Store validation outputs here

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        sequences, labels = batch
        outputs = self.model(sequences).squeeze(1)  # Shape: [batch_size]
        loss = self.loss_fn(outputs, labels)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        sequences, labels = batch
        outputs = self.model(sequences).squeeze(1)
        loss = self.loss_fn(outputs, labels)
        preds = torch.sigmoid(outputs) > 0.5
        acc = accuracy(preds, labels.int(), task="binary")
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        
        # Save outputs for epoch aggregation
        self.val_outputs.append(loss)
        return loss

    def configure_optimizers(self):
        # Define optimizer
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        
        # Define ReduceLROnPlateau scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode="min",            # Minimize the monitored metric
            patience=3,            # Number of epochs to wait before reducing LR
            factor=0.1,            # Factor by which to reduce LR
            verbose=True,          # Print messages when LR is reduced
            threshold=0.0001,      # Minimum change to qualify as an improvement
            cooldown=1             # Number of epochs to wait before resuming LR reduction
        )

        # Return as a dictionary
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch",      # Check metric every epoch
                "monitor": "val_loss",   # Metric to monitor
            },
        }


In [7]:
def train_and_evaluate(
    model,
    train_loader,
    val_loader,
    max_epochs=100,
    learning_rate=1e-3,
    early_stop_patience=5,
    log_dir="logs/",
    model_name="model",  # Pass the model name for logging
    dataset_stage="stage",  # Pass the dataset stage (easy, medium, hard, etc.)
):
    # Create a unique name for the logger based on the model and stage
    logger_name = f"{model_name}_{dataset_stage}"
    
    # TensorBoard logger
    tb_logger = TensorBoardLogger(save_dir=log_dir, name=logger_name)

    # Early stopping callback
    early_stopping = EarlyStopping(
        monitor="val_loss", patience=early_stop_patience, mode="min", verbose=True
    )

    # PyTorch Lightning Trainer
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1 if torch.cuda.is_available() else None,
        log_every_n_steps=1,
        callbacks=[early_stopping],
        logger=[tb_logger],  # Use the specific logger for this model and stage
    )

    # Initialize the SequenceClassifier with the provided model
    pl_module = SequenceClassifier(model, learning_rate)

    # Train and validate the model
    trainer.fit(pl_module, train_loader, val_loader)

In [8]:
# Paths for the datasets
easy_h5 = "/kaggle/input/pathfinder-easy-integer/merged_data.h5"
medium_h5 = "/kaggle/input/pathfinder-medium-integer/merged_data.h5"
hard_h5 = "/kaggle/input/pathfinder-hard-integer/merged_data.h5"

# Function to create train and validation datasets
def create_dataloaders(h5_file, batch_size=256, train_ratio=0.8):
    dataset = SequenceDataset(h5_file)
    total_size = len(dataset)
    train_size = int(total_size * train_ratio)
    val_size = total_size - train_size

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=3)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=3)

    return train_loader, val_loader
    

# Training models from scratch for each dataset
def train_from_scratch(model_classes, dataset_paths, log_dir="logs/", models_dir="./models/"):
    """
    Args:
        model_classes: List of tuples (model_type, model_class) to train.
        dataset_paths: List of tuples (dataset_path, stage) for training on datasets.
        log_dir: Directory to save logs.
        models_dir: Directory to save models.
    """
    os.makedirs(models_dir, exist_ok=True)

    for model_type, model_class in model_classes:
        print(f"\nStarting training for model type: {model_type.upper()}")

        for dataset_path, stage in dataset_paths:
            print(f"\nTraining {model_type.upper()} model on {stage.upper()} dataset...")

            # Create data loaders
            train_loader, val_loader = create_dataloaders(dataset_path)

            # Initialize model from scratch
            model = model_class()

            # Train and evaluate
            train_and_evaluate(
                model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                max_epochs=100,
                learning_rate=1e-3,
                early_stop_patience=5,
                log_dir=log_dir,
                model_name=model_type,
                dataset_stage=stage,
            )

            # Save the model
            save_path = os.path.join(models_dir, f"{model_type}_{stage}.pth")
            torch.save(model.state_dict(), save_path)
            print(f"Model saved at: {save_path}")
            del model  # Remove the current model
            torch.cuda.empty_cache()  # Free up all cached memory


In [9]:
# Define model types and datasets
model_classes = [
    ("rnn_normalization", RNNNormalizationModel),
    ("lstm_normalization", LSTMNormalizationModel),
    ("gru_normalization", GRUNormalizationModel),
    ("rnn", RNNModel),
    ("lstm", LSTMModel),
    ("gru", GRUModel),
]

datasets = [
    (easy_h5, "easy"),
    (medium_h5, "medium"),
    (hard_h5, "hard"),
]

# Call the training function for all models
train_from_scratch(model_classes, datasets)



Starting training for model type: RNN_NORMALIZATION

Training RNN_NORMALIZATION model on EASY dataset...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Model saved at: ./models/gru_hard.pth
